In [1]:
import requests
from lxml.etree import ElementTree as ET
from lxml import html

url = 'https://www.weather.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=109'

html_req = requests.get(url)

# 전달 받은 html정보 저장
tree = html.fromstring(html_req.content)

In [2]:
locations = tree.xpath('//location')

print(locations)

[<Element location at 0x23d55eba950>, <Element location at 0x23d55ebae50>, <Element location at 0x23d55eba540>, <Element location at 0x23d55ebaef0>, <Element location at 0x23d55ebaf40>, <Element location at 0x23d55ebaf90>, <Element location at 0x23d55f2d040>, <Element location at 0x23d55f2d090>, <Element location at 0x23d55f2d0e0>, <Element location at 0x23d55f2d130>, <Element location at 0x23d55f2d180>, <Element location at 0x23d55f2d1d0>, <Element location at 0x23d55f2d220>, <Element location at 0x23d55f2d270>, <Element location at 0x23d55f2d2c0>, <Element location at 0x23d55f2d310>, <Element location at 0x23d55f2d360>, <Element location at 0x23d55f2d3b0>, <Element location at 0x23d55f2d400>, <Element location at 0x23d55f2d450>, <Element location at 0x23d55f2d4a0>, <Element location at 0x23d55f2d4f0>, <Element location at 0x23d55f2d540>, <Element location at 0x23d55f2d590>, <Element location at 0x23d55f2d5e0>, <Element location at 0x23d55f2d630>, <Element location at 0x23d55f2d680>, 

In [3]:
import pandas as pd
import datetime

df = pd.DataFrame(columns=['도시','날짜','날씨','최고','최저'])

for location in locations:
    city = location.xpath('.//city/text()')[0]
    datas = location.xpath('.//data')
    
    for data in datas:
        date = str(data.xpath('.//tmef/text()')[0])
        date = datetime.datetime.strptime(date,'%Y-%m-%d %H:%M')
        #date = date.date()
        wf = data.xpath('.//wf/text()')[0]
        tmn = data.xpath('.//tmn/text()')[0]
        tmx = data.xpath('.//tmx/text()')[0]
        temp_dict = {
            '도시' : city,
            '날짜' : date,
            '날씨' : wf,
            '최고' : tmn,
            '최저' : tmx
        }
    
        df = df.append(temp_dict, ignore_index=True)
    
display(df)

,도시,날짜,날씨,최고,최저
0,서울,2021-07-26 00:00:00,맑음,26,35
1,서울,2021-07-26 12:00:00,맑음,26,35
2,서울,2021-07-27 00:00:00,맑음,25,35
3,서울,2021-07-27 12:00:00,맑음,25,35
4,서울,2021-07-28 00:00:00,맑음,25,34
...,...,...,...,...,...
450,여주,2021-07-30 00:00:00,구름많음,22,32
451,여주,2021-07-30 12:00:00,구름많음,22,32
452,여주,2021-07-31 00:00:00,구름많음,23,33
453,여주,2021-08-01 00:00:00,구름많음,24,31


In [4]:
df['날짜_연도'] = df['날짜'].dt.year
df['날짜_월'] = df['날짜'].dt.month
df['날짜_일'] = df['날짜'].dt.day
df.drop(['날짜'],axis=1,inplace=True)

In [5]:
df

,도시,날씨,최고,최저,날짜_연도,날짜_월,날짜_일
0,서울,맑음,26,35,2021,7,26
1,서울,맑음,26,35,2021,7,26
2,서울,맑음,25,35,2021,7,27
3,서울,맑음,25,35,2021,7,27
4,서울,맑음,25,34,2021,7,28
...,...,...,...,...,...,...,...
450,여주,구름많음,22,32,2021,7,30
451,여주,구름많음,22,32,2021,7,30
452,여주,구름많음,23,33,2021,7,31
453,여주,구름많음,24,31,2021,8,1


In [6]:
df_city= pd.get_dummies(df['도시'])
df_weather = pd.get_dummies(df['날씨'])

In [7]:
df = df.assign(**df_city)
df = df.assign(**df_weather)
#df = df.drop(['도시','날씨'],axis=1)
df

,도시,날씨,최고,최저,날짜_연도,날짜_월,날짜_일,가평,강화,고양,...,의정부,이천,인천,파주,평택,포천,하남,화성,구름많음,맑음
0,서울,맑음,26,35,2021,7,26,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,서울,맑음,26,35,2021,7,26,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,서울,맑음,25,35,2021,7,27,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,서울,맑음,25,35,2021,7,27,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,서울,맑음,25,34,2021,7,28,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,여주,구름많음,22,32,2021,7,30,0,0,0,...,0,0,0,0,0,0,0,0,1,0
451,여주,구름많음,22,32,2021,7,30,0,0,0,...,0,0,0,0,0,0,0,0,1,0
452,여주,구름많음,23,33,2021,7,31,0,0,0,...,0,0,0,0,0,0,0,0,1,0
453,여주,구름많음,24,31,2021,8,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [26]:
x_data = df.iloc[:,2:-1]
y_data = df.loc[:,'최저']

y_yeoju = df.iloc[454,2:-1]
y_seoul = df.iloc[0,2:-1]
y_godincheon = df.iloc[13,2:-1]

최고         26
최저         33
날짜_연도    2021
날짜_월        7
날짜_일       26
가평          0
강화          0
고양          0
과천          0
광명          0
광주          0
구리          0
군포          0
김포          0
남양주         0
동두천         0
백령도         0
부천          0
서울          0
성남          0
수원          0
시흥          0
안산          0
안성          0
안양          0
양주          0
양평          0
여주          0
연천          0
오산          0
용인          0
의왕          0
의정부         0
이천          0
인천          1
파주          0
평택          0
포천          0
하남          0
화성          0
구름많음        0
Name: 13, dtype: object


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge,Lasso,ElasticNet, LinearRegression
from sklearn.pipeline import Pipeline

x_train,x_test,y_train,y_test = train_test_split(x_data,y_data,test_size=0.2)

modelElastic = Pipeline([('scl', StandardScaler()), ('clf',ElasticNet()),])
param_value = {'clf__alpha': [0.001, 0.01, 1, 2, 3, 4],
              'clf__l1_ratio' : [0.1, 0.5, 1]}
gridSearch = GridSearchCV(modelElastic, param_grid = param_value, cv = 10, verbose = 1, scoring = 'r2')
gridSearch.fit(x_train, y_train)

gridSearch.best_params_

Fitting 10 folds for each of 18 candidates, totalling 180 fits


{'clf__alpha': 0.001, 'clf__l1_ratio': 1}

In [15]:
gridSearch.best_score_

0.9999994665553411

In [27]:
import numpy as np

y_predict_yeoju = gridSearch.predict(np.array(y_yeoju.values).reshape(1,-1))


y_predict_seoul = gridSearch.predict(np.array(y_seoul.values).reshape(1,-1))


y_predict_godincheon = gridSearch.predict(np.array(y_godincheon.values).reshape(1,-1))


print(f'여주 예측 {int(y_predict_yeoju[0])},서울 예측 {int(y_predict_seoul[0])}, 인천 예측 {int(y_predict_godincheon[0])}')

여주 예측 31,서울 예측 34, 인천 예측 32
